# EDA, Feature Engineering, and AutoML

In this lab, we will apply learned concepts from Day 3-4 lectures to perform data exploration, feature engineering, and autoML on a house sale dataset.

The goal of this assignment is to analyze 3 years’(2018-2020) house sales data provided by New York City (NYC) goverment and build regression model to predict house price. NYC has five boroughs, i.e., Bronx, Brooklyn, Manhattan, Queens and Staten Island. Sales of houses in each borough has been provided.


**Submission: submit via onq.** 


In [ ]:
# Import needed libraries. E.g., pandas, missingno, and sklearn
import pandas as pd
import glob
import sklearn
import missingno
from collections import Counter
import os
from IPython.display import display_html 
from scipy import stats
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import zipfile
from pprint import pprint
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
#!pip install -U scikit-learn
#!sudo apt-get install build-essential swig
#!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
#!pip install auto-sklearn

import autosklearn.regression


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


Task 1: Read dataset, merge data, and  perform basic data exploration. Note, you should briefely discuss the quality of dataset (missing values, duplicate entries, etc.) Also, in task 3, you would be asked to perform prediction on house sale price, thus it would be good to consider outliers related to this prediciton task when you perform data cleaning.

In [ ]:
# TODO write code to perform task 1

with zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/Datasets/Newyork cities.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')


path = r'/content/Newyork cities' # use your path

#Read all files
data = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, "*.csv"))))


#drop redundant columns
cols = list(range(21,42))
data.drop(data.columns[cols],axis=1,inplace=True)

'''Remove \n from column names'''
col = list(data.columns)
for i in range(len(col)):
    col[i] = col[i].replace('\n','')

#assign new columns names
data.columns = col
print(col)
data.sample(25)

In [ ]:
data.info()

### Check and handling the consistant duplicated values in this dataset.

In [ ]:
print(f"There are {data.duplicated().sum()} consistant duplicated values.")
data = data.drop_duplicates()
data.reset_index(drop=True, inplace=True)

print(f"There are {data.duplicated().sum()} consistant duplicated values now.")

In [ ]:
#most 15 frequented value in SALE data
data['SALE DATE'].value_counts()[:15]

### Check and handling the missing values values in this dataset.

In [ ]:
if data.isna().values.any() == True:
    #this condidtion will be evaluated.
    print("There are missing values in this dataframe")
else:
    print("There aren't missing values in this dataframe")
total_miss = data.isnull().sum()
percent_miss = (total_miss/data.isnull().count()*100)

# sort attributes by missing value ratio
missing_data = pd.DataFrame({'Total missing':total_miss,'% missing':percent_miss})
missing_data.sort_values(by='Total missing',ascending=False)

In [ ]:
missingno.matrix(data)

'BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'BLOCK', 'LOT', 'ADDRESS', 'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE', 'SALE PRICE', 'SALE DATE' features follow the same patterns in missing values<br>

There are only 1 missing value after dropping the duplicates so they could be follow the **MCAR** mechanism.

There are only 17 missing value in ZIP CODE feature after dropping the duplicates so it could be follow the **MCAR** mechanism.


YEAR BUILT and APARTMENT NUMBER doesn't follow any pattern so they are maybe following **MCAR** mechanism pattern and they have a lot of missing values so I'll drop the entire column.

I'll drop the missing values in the features that follows **MCAR** mechanism excpt for **YEAR BUILT** and **APARTMENT NUMBER**.

In [ ]:
data = data.dropna(axis='rows',subset=['BOROUGH', 'NEIGHBORHOOD','ZIP CODE', 'BUILDING CLASS CATEGORY', 'BLOCK', 'LOT', 'ADDRESS', 'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE', 'SALE PRICE', 'SALE DATE'])

I'll drop the EASE-MENT and APARTMENT NUMBER feature since the most of their data are missing values and they don't have any pattern with another feature.

In [ ]:
data = data.drop(['EASE-MENT', 'APARTMENT NUMBER'],axis=1)

In [ ]:
missingno.matrix(data)

In [ ]:
missingno.bar(data)

In [ ]:
missingno.heatmap(data)

In [ ]:
missingno.dendrogram(data)

'TAX CLASS AS OF FINAL ROLL 18/19', 'BUILDING CLASS AS OF FINAL ROLL 18/19' are following the same patterns in missing values.

'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS', 'LAND SQUARE FEET', 'GROSS SQUARE FEET' features follow the same patterns in missing values.

### Let's analyiz and deal with missing values at **'TAX CLASS AS OF FINAL ROLL 18/19', 'BUILDING CLASS AS OF FINAL ROLL 18/19'** features.

In [ ]:
data['TAX CLASS AS OF FINAL ROLL 18/19'].value_counts()

In [ ]:
#most 15 frequented value in BUILDING CLASS AS OF FINAL ROLL 18/19
data['BUILDING CLASS AS OF FINAL ROLL 18/19'].value_counts()[:15]

In [ ]:
#you can change the value of TAX CLASS AS OF FINAL ROLL 18/19 with BUILDING CLASS AS OF FINAL ROLL 18/19, since they are follow the same patterns. 

#check how many missing values in TAX CLASS AS OF FINAL ROLL 18/19 or BUILDING CLASS AS OF FINAL ROLL 18/19 at each BOROUGH value
analysis_1 = pd.DataFrame.from_dict(Counter(data['BOROUGH'].loc[data['TAX CLASS AS OF FINAL ROLL 18/19'].isna()]), orient='index', columns = ['number of missings']).reset_index().sort_values('number of missings',ascending =False)
analysis_1.rename(columns={'index': 'BOROUGH'}, inplace = True)

#check how many missing values in TAX CLASS AS OF FINAL ROLL 18/19 or BUILDING CLASS AS OF FINAL ROLL 18/19 at each NEIGHBORHOOD value
analysis_2 = pd.DataFrame.from_dict(Counter(data['NEIGHBORHOOD'].loc[data['TAX CLASS AS OF FINAL ROLL 18/19'].isna()]), orient='index', columns = ['number of missings']).reset_index().sort_values('number of missings',ascending =False)[:30]
analysis_2.rename(columns={'index': 'NEIGHBORHOOD'}, inplace = True)

#check how many missing values in TAX CLASS AS OF FINAL ROLL 18/19 or BUILDING CLASS AS OF FINAL ROLL 18/19 at each BUILDING CLASS CATEGORY value
analysis_3 = pd.DataFrame.from_dict(Counter(data['BUILDING CLASS CATEGORY'].loc[data['TAX CLASS AS OF FINAL ROLL 18/19'].isna()]), orient='index', columns = ['number of missings']).reset_index().sort_values('number of missings',ascending =False)[:30]
analysis_3.rename(columns={'index': 'BUILDING CLASS CATEGORY'}, inplace = True)

#check how many missing values in TAX CLASS AS OF FINAL ROLL 18/19 or BUILDING CLASS AS OF FINAL ROLL 18/19 at each BLOCK value
analysis_4 = pd.DataFrame.from_dict(Counter(data['BLOCK'].loc[data['TAX CLASS AS OF FINAL ROLL 18/19'].isna()]), orient='index', columns = ['number of missings']).reset_index().sort_values('number of missings',ascending =False)
analysis_4.rename(columns={'index': 'BLOCK'}, inplace = True)

analysis_1_styler = analysis_1.style.set_table_attributes("style=display:inline").set_caption('number of missings at each BOROUGH value')
analysis_2_styler = analysis_2.style.set_table_attributes("style=display:inline").set_caption('number of missings at each NEIGHBORHOOD value')
analysis_3_styler = analysis_3.style.set_table_attributes("style=display:inline").set_caption('number of missings at each BUILDING CLASS CATEGORY value')
analysis_4_styler = analysis_4.style.set_table_attributes("style=display:inline").set_caption('number of missings at each BLOCK value')

display_html(analysis_1_styler._repr_html_()+analysis_2_styler._repr_html_()+analysis_3_styler._repr_html_()+analysis_4_styler._repr_html_(), raw=True)

If you may notice from the above table most of the null values in **TAX CLASS AS OF FINAL ROLL 18/19** and **BUILDING CLASS AS OF FINAL ROLL 18/19** happened when the values in **BLOCK** were **1296.00000** and the values in the **NEIGHBORHOOD** features was **MIDTOWN WEST** also there are more than **200** missing values happened when the values in the **BUILDING CLASS CATEGORY** was **13 CONDOS - ELEVATOR APARTMENTS** so based on these observed data the missing values in these two features follow **MAR** mechanism.

Since the missing values in these two features is not a lot I'll fill the missings with most frequented values.

In [ ]:
data['TAX CLASS AS OF FINAL ROLL 18/19'].fillna(data['TAX CLASS AS OF FINAL ROLL 18/19'].mode()[0],inplace=True)
data['BUILDING CLASS AS OF FINAL ROLL 18/19'].fillna(data['BUILDING CLASS AS OF FINAL ROLL 18/19'].mode()[0],inplace=True)

### Let's analyiz and deal with missing values at **'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS', 'LAND SQUARE FEET', 'GROSS SQUARE FEET'** features.

In [ ]:
#you can change the value of RESIDENTIAL UNITS with BUILDING CLASS AS OF FINAL ROLL 18/19, since they are follow the same patterns. 

#check how many missing values in RESIDENTIAL UNITS or BUILDING CLASS AS OF FINAL ROLL 18/19 at each BOROUGH value
analysis_5 = pd.DataFrame.from_dict(Counter(data['BOROUGH'].loc[data['RESIDENTIAL UNITS'].isna()]), orient='index', columns = ['number of missings']).reset_index().sort_values('number of missings',ascending =False)
analysis_5.rename(columns={'index': 'BOROUGH'}, inplace = True)

#check how many missing values in RESIDENTIAL UNITS or BUILDING CLASS AS OF FINAL ROLL 18/19 at each NEIGHBORHOOD value
analysis_6 = pd.DataFrame.from_dict(Counter(data['NEIGHBORHOOD'].loc[data['RESIDENTIAL UNITS'].isna()]), orient='index', columns = ['number of missings']).reset_index().sort_values('number of missings',ascending =False)[:30]
analysis_6.rename(columns={'index': 'NEIGHBORHOOD'}, inplace = True)

#check how many missing values in RESIDENTIAL UNITS or BUILDING CLASS AS OF FINAL ROLL 18/19 at each BUILDING CLASS CATEGORY value
analysis_7 = pd.DataFrame.from_dict(Counter(data['BUILDING CLASS CATEGORY'].loc[data['RESIDENTIAL UNITS'].isna()]), orient='index', columns = ['number of missings']).reset_index().sort_values('number of missings',ascending =False)[:30]
analysis_7.rename(columns={'index': 'BUILDING CLASS CATEGORY'}, inplace = True)

#check how many missing values in RESIDENTIAL UNITS or BUILDING CLASS AS OF FINAL ROLL 18/19 at each BLOCK value
analysis_8 = pd.DataFrame.from_dict(Counter(data['TAX CLASS AS OF FINAL ROLL 18/19'].loc[data['RESIDENTIAL UNITS'].isna()]), orient='index', columns = ['number of missings']).reset_index().sort_values('number of missings',ascending =False)
analysis_8.rename(columns={'index': 'BUILDING CLASS AS OF FINAL ROLL 18/19'}, inplace = True)

analysis_5_styler = analysis_5.style.set_table_attributes("style=display:inline").set_caption('number of missings at each BOROUGH value')
analysis_6_styler = analysis_6.style.set_table_attributes("style=display:inline").set_caption('number of missings at each NEIGHBORHOOD value')
analysis_7_styler = analysis_7.style.set_table_attributes("style=display:inline").set_caption('number of missings at each BUILDING CLASS CATEGORY value')
analysis_8_styler = analysis_8.style.set_table_attributes("style=display:inline").set_caption('number of missings at each BLOCK value')

display_html(analysis_5_styler._repr_html_()+analysis_6_styler._repr_html_()+analysis_7_styler._repr_html_()+analysis_8_styler._repr_html_(), raw=True)

If you may notice from the above table most of the null values in **RESIDENTIAL UNITS** feature happened when the values in **BUILDING CLASS AS OF FINAL ROLL 18/19** feature were **2** and so it follows **MAR** mechanism and since **'COMMERCIAL UNITS', 'TOTAL UNITS', 'LAND SQUARE FEET', 'GROSS SQUARE FEET'** follow the same pattern with in missing values with **RESIDENTIAL UNITS** so they also follow **MAR** mechanism.

#### Handle **TOTAL UNITS** feature

note: this feature are string feature but it has integer numerical values.

In [ ]:
#most 20 frequented values in RESIDENTIAL UNITS
data['TOTAL UNITS'] = data['TOTAL UNITS'].str.replace(',','.')

data['TOTAL UNITS'] = pd.to_numeric(data['TOTAL UNITS'])

I see that I should replace the nan values in these two features with the median each feature.

In [ ]:
data['TOTAL UNITS'] = data['TOTAL UNITS'].fillna(int(data['TOTAL UNITS'].median()))

data['TOTAL UNITS'].value_counts()[:10]

### Handle **'LAND SQUARE FEET', 'GROSS SQUARE FEET'** features.
note: these features are string features but they have integer numerical values.

In [ ]:
print(data['LAND SQUARE FEET'].value_counts()[:10])
print(data['GROSS SQUARE FEET'].value_counts()[:10])

data['LAND SQUARE FEET'] = data['LAND SQUARE FEET'].str.replace(',','')
data['GROSS SQUARE FEET'] = data['GROSS SQUARE FEET'].str.replace(',','')

data['LAND SQUARE FEET'] = pd.to_numeric(data['LAND SQUARE FEET'])
data['GROSS SQUARE FEET'] = pd.to_numeric(data['GROSS SQUARE FEET'])


print(data['LAND SQUARE FEET'].value_counts()[0] / len(data) * 100)

I saw that I should replace null values in these features with the mode since they are representing most of the data in these features.

In [ ]:
data['LAND SQUARE FEET'] = data['LAND SQUARE FEET'].fillna(data['LAND SQUARE FEET'].mode()[0])
data['GROSS SQUARE FEET'] = data['GROSS SQUARE FEET'].fillna(data['GROSS SQUARE FEET'].mode()[0])

In [ ]:
missingno.matrix(data)

### I won't handle **COMMERCIAL UNITS** and **RESIDENTIAL UNITS** and I'll explain why in the feature engineering step.

### Handle **YEAR BUILT** feature

note: there are 0.0 value in this feature I considered them as noisy data and I'll replace them with the median of the data also as Null values.

In [ ]:
print(data['YEAR BUILT'].value_counts())

In [ ]:
data['YEAR BUILT'] = data['YEAR BUILT'].fillna(int(data['YEAR BUILT'].median()))
data['YEAR BUILT'] = data['YEAR BUILT'].replace(0.0,int(data['YEAR BUILT'].median()))

### Feature selection step related to missing values

I'll drop **RESIDENTIAL UNITS** and **COMMERCIAL UNITS** while they are have missing values since the **TOTAL UNITS** feature equal their sum, and I'll drop **ADDRESS** since it has a lot of unique values and I see that **NEIGHBORHOOD** represent its values.

In [ ]:
data = data.drop(['RESIDENTIAL UNITS','COMMERCIAL UNITS','ADDRESS'], axis = 1)

### Check if there are still any missing 

In [ ]:
if data.isna().values.any() == True:
    #this condidtion will be evaluated.
    print("There are missing values in this dataframe")
else:
    print("There aren't missing values in this dataframe")
total_miss = data.isnull().sum()
percent_miss = (total_miss/data.isnull().count()*100)

# sort attributes by missing value ratio
missing_data = pd.DataFrame({'Total missing':total_miss,'% missing':percent_miss})
missing_data.sort_values(by='Total missing',ascending=False)

### Reset indices of the data

In [ ]:
data.reset_index(drop= True, inplace = True)

### Check and handling the outliers for some numerical features.

If the feature has more than 1200 values as outliers I won't drop them otherwise I'll normalize this feature.

In [ ]:
sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(11,6)})

def show_boxplot(df,col):
    outlier = sns.boxplot(x=df[col])
    outlier.set(title = f"{col} feature outliers analysis.")

def analysis_and_remove_outliers(df, num_cols):
    
    for column in num_cols:
        Q1 = np.percentile(df[column], 20, interpolation = 'midpoint')
                      
        Q3 = np.percentile(df[column], 80, interpolation = 'midpoint')
        IQR = Q3 - Q1
                      
        # Upper bound
        upper_bound = np.where(df[column] > (Q3+1.5*IQR))
        # Lower bound
        lower_bound = np.where(df[column] < (Q1-1.5*IQR))

        num_of_outliers = len(upper_bound[0]) + len(lower_bound[0])
                      
        print(f"number of outliers at '{column} feature is: {num_of_outliers} value.")
                      
        df.loc[upper_bound[0], column]= np.nan
        df.loc[lower_bound[0], column]= np.nan

    df = df.dropna()
    return df

first I should convert **SALE PRICE** feature to numerical feature since it has numerical values but in string form.

In [ ]:
data['SALE PRICE'] = data['SALE PRICE'].str.replace(',','')
data['SALE PRICE'] = data['SALE PRICE'].str.replace('$','')
data['SALE PRICE'] = pd.to_numeric(data['SALE PRICE'])

In [ ]:
num_cols = list(data.select_dtypes(include=['int','float']))
#We won't handle the outliers in the Rating column although we it doesn't has any outliers but it's the output column.
num_cols.remove('SALE PRICE')
#We will change the data type of this column in the future to category.
num_cols.remove('ZIP CODE')
print("Old shape after droping all outliers values in all numeric columns: ", data.shape)

In [ ]:
show_boxplot(data,num_cols[0])
print("No outliers founded")

In [ ]:
show_boxplot(data,num_cols[1])

#### Analysis and Handling outliers in the numeric feature.

In [ ]:
data = analysis_and_remove_outliers(data, num_cols)

### The shape of the data after handling duplication, missing and outliers data

In [ ]:
#new shape of the data 
data.shape

### Clean format of the **SALE DATE**

In [ ]:
data['SALE DATE'] = pd.to_datetime(data['SALE DATE'])
data['SALE DATE'] = data['SALE DATE'].astype(str)

Task 2: Data exploration using data visualization.
Raise two questions that can be answered by performing data visualization. Briefely mention why you think this question would be interesting to whom (who is your audience). Think about the EDA principals.

1. What's the top 5 Building class category at NY state?

Answering question will help new people that would move to NY state and would help them to choose a suitable city, also will help Real estate men and brokers to determine what's the most Building class category at NY state and they would compare it with other states.

2. Do the number of Total units increased over years?

Answering this question will help economists and data analysis to determine why in a specific year the number of units was less or more than usual, and it will determine the GPT of the country at this year.

Question 2.1: What's the top 5 Building class category at NY state?

Answering question will help new people that would move to NY state and would help them to choose a suitable city, also will help Real estate men and brokers to determine what's the most Building class category at NY state and they would compare it with other states.

In [ ]:
# TODO write code to answer Q2.1
data['BUILDING CLASS CATEGORY'].value_counts()[:5].plot(kind='pie')

In [ ]:
#answer Q2.1
data['BUILDING CLASS CATEGORY'].value_counts()[:8].plot(kind='bar')

Question 2.2: {put your raised question here.}

In [ ]:
num_cols

In [ ]:
# TODO write code to answer Q2.2

new_1 = data.loc[(data['YEAR BUILT']>=1950)]['TOTAL UNITS'].mean()
new_2 = data.loc[(data['YEAR BUILT']<1950)]['TOTAL UNITS'].mean()
print(new_1)
print(new_2)
df = data.groupby('YEAR BUILT').count()['TOTAL UNITS']
df

From above test the number of units in the older years was near to total units in the newer years.

Task 3: Data Exploration via Statistical Test
Raise one question that can be answered by performing hypothesis test. Briefely mention why you think this question would be interesting to whom (who is your audience). Also mention which statistical test you would choose and why.

Null hypothesis: Number of Total units are higher in the older years in NY state? i.e <1950 the number total units became more than 1000.

Alternative hypothesis: Number of Total units in the older years is near to the number of total units in the newer years in NY state?

In [ ]:
# TODO write code to perform task 3
new_1 = data.loc[(data['YEAR BUILT']>=1980)]['TOTAL UNITS']
new_2 = data.loc[(data['YEAR BUILT']<1980)]['TOTAL UNITS']

ttest,significance_level = ttest_ind(new_1,new_2,equal_var=False) #Assume samples have equal variance
print("p-value: ",significance_level)
print(f"t-test value: {ttest}")

if significance_level > 0.05:
  print("we accept null hypothesis")
else:
  print("we reject null hypothesis")

findings: After I tested hypothese, The exprement reject my assumtion and you can see from the mean of two values that they aren't have the much difference.

Task 4: Feature Engineering. If we would like to predict the house sale price. Analyze the scale of each attribute and determine which ones you would transfer (e.g., cateogorical features). Discuss how you plan to select important features.

In [ ]:
# TODO for Task 4
data.describe()

In [ ]:
#check the distribution of each numerical column.
for column in data.columns:
    if data[column].dtype != object:
        fig = plt.figure(figsize=(10,10))
        sns.displot(data[column], kind="kde")
        plt.close(fig)

### 1: Normalize numerical features that have outliers.

In [ ]:
print(data['TOTAL UNITS'].max())
print(data['TOTAL UNITS'].min())

In [ ]:
scaler = MinMaxScaler()

data[['BLOCK','GROSS SQUARE FEET','TOTAL UNITS','LAND SQUARE FEET','TAX CLASS AT TIME OF SALE']] = scaler.fit_transform(data[['BLOCK','GROSS SQUARE FEET','TOTAL UNITS','LAND SQUARE FEET','TAX CLASS AT TIME OF SALE']])

In [ ]:
print(data['TOTAL UNITS'].max())
print(data['TOTAL UNITS'].min())

### Correlation analysis

In [ ]:
plt.figure(figsize = (20,7))
sns.heatmap(data.corr(), annot = True, cmap="rainbow")
plt.show()

Since there aren't highly correlated feature so I won't drop any columns anymore.

In [ ]:
data.head(5)

### 2: Convert the dtype of object features **('NEIGHBORHOOD','BUILDING CLASS CATEGORY','BUILDING CLASS AS OF FINAL ROLL 18/19','TAX CLASS AS OF FINAL ROLL 18/19','BUILDING CLASS AT TIME OF SALE')** to categorical data type and then convert them to labels using label encoder but that isn't right solutions and I had to use it since automl took a lot of ram if I converted them to one hot encoded data and the model crashs.

In [ ]:
data[['NEIGHBORHOOD','BUILDING CLASS CATEGORY','BUILDING CLASS AS OF FINAL ROLL 18/19','TAX CLASS AS OF FINAL ROLL 18/19','BUILDING CLASS AT TIME OF SALE']] = data[['NEIGHBORHOOD','BUILDING CLASS CATEGORY','BUILDING CLASS AS OF FINAL ROLL 18/19','TAX CLASS AS OF FINAL ROLL 18/19','BUILDING CLASS AT TIME OF SALE']].astype('category')
en = LabelEncoder()

data['NEIGHBORHOOD'] = en.fit_transform(data['NEIGHBORHOOD'])
data['BUILDING CLASS CATEGORY'] = en.fit_transform(data['BUILDING CLASS CATEGORY'])
data['BUILDING CLASS AS OF FINAL ROLL 18/19'] = en.fit_transform(data['BUILDING CLASS AS OF FINAL ROLL 18/19'])
data['TAX CLASS AS OF FINAL ROLL 18/19'] = en.fit_transform(data['TAX CLASS AS OF FINAL ROLL 18/19'])
data['BUILDING CLASS AT TIME OF SALE'] = en.fit_transform(data['BUILDING CLASS AT TIME OF SALE'])
data['SALE DATE'] = en.fit_transform(data['SALE DATE'])

#data = pd.get_dummies(data, columns=['NEIGHBORHOOD','BUILDING CLASS CATEGORY','BUILDING CLASS AS OF FINAL ROLL 18/19','TAX CLASS AS OF FINAL ROLL 18/19','BUILDING CLASS AT TIME OF SALE','SALE DATE'])

In [ ]:
data.shape

### Split the data into train and test data

In [ ]:
y = data['SALE PRICE']
data = data.drop(['SALE PRICE'],axis = 1)
X_train, X_test, y_train, y_test = train_test_split(data, y,test_size = 0.3 , random_state=1)

X_train.shape

### Simple Linear Regression model

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train,y_train)

y_predicted = model.predict(X_test)

#evaluate the model

error = mean_squared_error(y_test, y_predicted)

error

Why this huge MSE error?

I think that's because this problem is not linearly seperable, so I should use more complex model.

Task 5: AutoML

Using Auto-sklearn to explore performance of one state-of-the-art autoML tool on the given data (after your previous preprocessing). Optional: compare with performing autosklearn on raw data.
ref. https://automl.github.io/auto-sklearn/master/examples/20_basic/example_regression.html

### Build and fit a regressor


In [ ]:
# TODO for Task 5
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=1500,
    per_run_time_limit=200,
    n_jobs = -1
)
automl.fit(X_train, y_train)

In [ ]:
print(automl.leaderboard())

In [ ]:
#Print the final ensemble constructed by auto-sklearn¶
pprint(automl.show_models(), indent=4)

In [ ]:
#Get the Score of the final ensemble¶
train_predictions = automl.predict(X_train)
print("Train R2 score:", sklearn.metrics.r2_score(y_train, train_predictions))
test_predictions = automl.predict(X_test)
print("Test R2 score:", sklearn.metrics.r2_score(y_test, test_predictions))

In [ ]:
#Plot the predictions¶
plt.scatter(train_predictions, y_train, label="Train samples", c='#d95f02')
plt.scatter(test_predictions, y_test, label="Test samples", c='#7570b3')
plt.xlabel("Predicted value")
plt.ylabel("True value")
plt.legend()
plt.tight_layout()
plt.show()

### From the above results using AutoML was much better than LinearRegression and the best Regressor was Gradientboosting regressor with less much cost than LinearRegressor.

note: the result may vary at each time.